# Tomato Disease Diagnosis

In [1]:
from fastai.vision.all import *
import params
from fastcore.all import *
import wandb

In [2]:
Path.BASE_PATH = path = Path("/notebooks/tomatodiagnosis/data/")

In [3]:
path.ls()

(#10) [Path('Tomato_Yellow_Leaf_Curl_Virus'),Path('healthy'),Path('Late_blight'),Path('Septoria_leaf_spot'),Path('Leaf_Mold'),Path('Spider_mites'),Path('Tomato_mosaic_virus'),Path('Early_blight'),Path('Bacterial_spot'),Path('Target_Spot')]

Here we define several functions to help us process the data and upload it as a Table to W&B.

In [4]:
def label_func(fname):
    return (fname.parent.name)

def _create_table(image_files, class_labels):
    "Create a table with the dataset"
    labels = [str(class_labels[_lab]) for _lab in list(class_labels)]
    table = wandb.Table(columns=["File_Name", "Images", "Split", "Label", "Dataset"])
    
    for i, image_file in progress_bar(enumerate(image_files), total=len(image_files)):
        image = Image.open(image_file)
        label = label_func(image_file) 
        table.add_data(
            str(image_file.parent.name) + '/' + image_file.name, 
            wandb.Image(image),
            "None",
            label,
            "Tomato"
        )    
    return table

In [5]:
run = wandb.init(project=params.WANDB_PROJECT, job_type="upload")
raw_data_at = wandb.Artifact(params.RAW_DATA_AT, type="raw_data")

wandb: Currently logged in as: solab5. Use `wandb login --relogin` to force relogin


Let's add the images

In [6]:
raw_data_at.add_dir(path/'Tomato_Yellow_Leaf_Curl_Virus', name='Tomato_Yellow_Leaf_Curl_Virus')
raw_data_at.add_dir(path/'healthy', name='healthy')
raw_data_at.add_dir(path/'Late_blight', name='Late_blight')
raw_data_at.add_dir(path/'Septoria_leaf_spot', name='Septoria_leaf_spot')
raw_data_at.add_dir(path/'Leaf_Mold', name='Leaf_Mold')
raw_data_at.add_dir(path/'Spider_mites', name='Spider_mites')
raw_data_at.add_dir(path/'Tomato_mosaic_virus', name='Tomato_mosaic_virus')
raw_data_at.add_dir(path/'Early_blight', name='Early_blight')
raw_data_at.add_dir(path/'Bacterial_spot', name='Bacterial_spot')
raw_data_at.add_dir(path/'Target_Spot', name='Target_Spot')

wandb: Adding directory to artifact (/notebooks/tomatodiagnosis/data/Tomato_Yellow_Leaf_Curl_Virus)... Done. 6.9s
wandb: Adding directory to artifact (/notebooks/tomatodiagnosis/data/healthy)... Done. 2.3s
wandb: Adding directory to artifact (/notebooks/tomatodiagnosis/data/Late_blight)... Done. 2.9s
wandb: Adding directory to artifact (/notebooks/tomatodiagnosis/data/Septoria_leaf_spot)... Done. 3.2s
wandb: Adding directory to artifact (/notebooks/tomatodiagnosis/data/Leaf_Mold)... Done. 1.4s
wandb: Adding directory to artifact (/notebooks/tomatodiagnosis/data/Spider_mites)... Done. 2.8s
wandb: Adding directory to artifact (/notebooks/tomatodiagnosis/data/Tomato_mosaic_virus)... Done. 0.5s
wandb: Adding directory to artifact (/notebooks/tomatodiagnosis/data/Early_blight)... Done. 1.3s
wandb: Adding directory to artifact (/notebooks/tomatodiagnosis/data/Bacterial_spot)... Done. 3.0s
wandb: Adding directory to artifact (/notebooks/tomatodiagnosis/data/Target_Spot)... Done. 1.7s


Let's get the file names of images in our dataset and use the function we defined above to create a W&B Table.

In [7]:
image_files = get_image_files(path)

In [8]:
table = _create_table(image_files, params.BDD_CLASSES)

Finally, we will add the Table to our Artifact, log it to W&B and finish our run.

In [9]:
raw_data_at.add(table, "eda_table3")

ArtifactManifestEntry(path='eda_table3.table.json', digest='HUsEQatGhWf0oL1sVlXrIA==', ref=None, birth_artifact_id=None, size=5028242, extra={}, local_path='/root/.local/share/wandb/artifacts/staging/tmpqhnau613')

In [10]:
run.log_artifact(raw_data_at)
run.finish()